In [1]:
import pandas as pd
import numpy as np

In [2]:
file_path = "otodom/geocoded/otodom_geocoded.csv"
df = pd.read_csv(file_path, sep="\t")
df.shape

(34931, 34)

In [21]:
df.rename(columns = {"Url": "URL"}, inplace=True)

In [3]:
df["Cena"] = df["Cena"].str[:-3]
df["Cena"] = df["Cena"].str.replace(",", ".")
df["Cena"] = df["Cena"].str.replace(" ", "")
df["Cena"] = df["Cena"].astype(float)

In [4]:
df["Powierzchnia"] = df["Powierzchnia"].str[:-3]
df["Powierzchnia"] = df["Powierzchnia"].str.replace(",", ".")
df["Powierzchnia"] = df["Powierzchnia"].str.replace(" ", "")
df["Powierzchnia"] = df["Powierzchnia"].astype(float)

In [5]:
df["Liczba pokoi"].replace({"wiecej niz 10": 15}, inplace=True)
df["Liczba pokoi"] = df["Liczba pokoi"].astype(float)

In [6]:
from typing import Optional

def convert_string_to_floor(floor: str, max_floor: Optional[str] = None):
    if floor == "parter":
        return 0
    elif floor == "suterena":
        return -1
    elif floor == "poddasze":
        return int(max_floor)
    else:
        return int(floor)

def prepare_floor(row: pd.Series):
    floor = row["Pietro"]
    floor = floor.replace(">", "")
    floor_elements = floor.split("/")
    if len(floor_elements) == 2:
        your_floor = floor_elements[0].strip()
        max_floor = floor_elements[1].strip()
        your_floor = convert_string_to_floor(your_floor, max_floor)
        max_floor = int(max_floor)
           
    if len(floor_elements) == 1:
        your_floor = floor_elements[0].strip()
        if your_floor == "Zapytaj":
            your_floor = None
            max_floor = None
        else:
            your_floor = convert_string_to_floor(your_floor)
        max_floor = None
    
    row["pietro mieszkania"] = your_floor
    row["maksymalne pietro"] = max_floor
    return row
    
df = df.apply(prepare_floor, axis=1)

In [7]:
df["polozenie mieszkania"] = df["pietro mieszkania"] / df["maksymalne pietro"]

In [8]:
df["Czynsz"] = df["Czynsz"].str.replace("zl", "")
df["Czynsz"] = df["Czynsz"].str.replace("EUR", "")
df["Czynsz"] = df["Czynsz"].str.replace(",", ".")
df["Czynsz"] = df["Czynsz"].str.replace(" ", "")
df.loc[(df["Czynsz"] == "Zapytaj") | (df["Czynsz"].isna()), "Czynsz"] = np.nan
df["Czynsz"] = df["Czynsz"].astype(float)


In [9]:
df.loc[df["Stan wykonczenia"] == "Zapytaj", "Stan wykonczenia"] = "Brak danych"
df.loc[df["Stan wykonczenia"].isna(), "Stan wykonczenia"] = "Brak danych"

In [19]:
df.rename(columns = {"Balkon / ogrod / taras": "Powierzchnia dodatkowa"}, inplace=True)

In [11]:
df.loc[(df["Miejsce parkingowe"] == "Zapytaj")|(df["Miejsce parkingowe"].isna()), "Miejsce parkingowe"] = 0
df.loc[(df["Miejsce parkingowe"] != "Zapytaj")&(~df["Miejsce parkingowe"].isna()), "Miejsce parkingowe"] = 1

In [12]:
df.loc[(df["Rynek"] == "Zapytaj")|(df["Rynek"].isna()), "Rynek"] = "Brak danych"

In [13]:
df.loc[(df["Rok budowy"] == "brak informacji") | (df["Rok budowy"].isna()), "Rok budowy"] = "Brak danych"

In [14]:
df.loc[(df["Winda"] == "nie") | (df["Winda"].isna()), "Winda"] = 0
df.loc[df["Winda"] == "tak", "Winda"] = 1
df["Winda"] = df["Winda"].astype(float)

In [15]:
df.drop(columns=[
    "Adres",
    "Tytul",
    "Obsluga zdalna",
    "Forma wlasnosci",
    "Ogrzewanie",
    "Typ ogloszeniodawcy",
    "Dostepne od",
    "Rodzaj zabudowy", 
    "Okna",
    "Media",
    "Zabezpieczenia",
    "Wyposazenie",
    "Informacje dodatkowe",
    "Material budynku"
], inplace=True)

In [22]:
df.columns

Index(['Miasto', 'URL', 'Cena', 'Powierzchnia', 'Liczba pokoi', 'Pietro',
       'Czynsz', 'Stan wykonczenia', 'Powierzchnia dodatkowa',
       'Miejsce parkingowe', 'Rynek', 'Rok budowy', 'Winda', 'latitude',
       'longitude', 'formatted_address', 'result_confidence', 'suburb',
       'building_category', 'result_type', 'pietro mieszkania',
       'maksymalne pietro', 'polozenie mieszkania'],
      dtype='object')

In [17]:
df.shape

(34931, 23)

In [24]:
df.dtypes

Miasto                     object
URL                        object
Cena                      float64
Powierzchnia              float64
Liczba pokoi              float64
Pietro                     object
Czynsz                    float64
Stan wykonczenia           object
Powierzchnia dodatkowa     object
Miejsce parkingowe         object
Rynek                      object
Rok budowy                 object
Winda                       int32
latitude                  float64
longitude                 float64
formatted_address          object
result_confidence         float64
suburb                     object
building_category          object
result_type                object
pietro mieszkania         float64
maksymalne pietro         float64
polozenie mieszkania      float64
dtype: object

In [ ]:
df_preprocesed = df[["Miasto", "Cena", "Powierzchnia", "Liczba pokoi", "pietro mieszkania", "maksymalne pietro", "polozenie mieszkania", "Czynsz", "Forma wlasnosci", "Stan wykonczenia", "Miejsce parkingowe", "Ogrzewanie", "Rynek", "Rok budowy", "Winda", "latitude", "longitude"]]

In [ ]:
df_preprocesed.head()

In [ ]:
df_preprocesed.dtypes

In [ ]:
from sklearn.model_selection import train_test_split
# columns = ["Cena", "Powierzchnia", "Liczba pokoi", "pietro mieszkania", "maksymalne pietro", "polozenie mieszkania", "Rok budowy", "Winda", "Czynsz", "latitude", "longitude"]
columns = ["Cena", "Powierzchnia", "Liczba pokoi", "pietro mieszkania", "maksymalne pietro", "polozenie mieszkania", "Rok budowy", "Winda", "Czynsz", "latitude", "longitude"]
df_modeling = df_preprocesed[columns]
df_modeling = df_modeling.fillna(0)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df_modeling.iloc[:, 1:], df_modeling["Cena"], test_size=0.2, random_state=42)

In [ ]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression().fit(x_train, y_train)

In [ ]:
regressor.score(x_train, y_train)

In [ ]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
r2 = r2_score(y_test, regressor.predict(x_test))
mse = mean_squared_error(y_test, regressor.predict(x_test))
print(r2)
print(mse)

In [ ]:
from sklearn.tree import DecisionTreeRegressor
regressor = DecisionTreeRegressor(random_state=0).fit(x_train, y_train)
regressor.score(x_train, y_train)

In [ ]:
r2 = r2_score(y_test, regressor.predict(x_test))
mse = mean_squared_error(y_test, regressor.predict(x_test))
print(r2)
print(mse)

In [ ]:
from sklearn.feature_selection import SelectKBest, RFE, f_regression

bestfeatures = SelectKBest(score_func=f_regression, k='all')
fit = bestfeatures.fit(x_train, y_train)
print(fit.scores_)
dfscores = pd.DataFrame(np.around(fit.scores_, 2))
dfcolumns = pd.DataFrame(x_train.columns)
#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  #naming the dataframe columns
print(featureScores.nlargest(30,'Score'))  
featureScores.head(100)